## Initialize a Spark Session

In [1]:
# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/phuongnam/Documents/Ki2_nam3/Big_Data/spark-3.5.1-bin-hadoop3"
os.environ["HADOOP_HOME"] = "/home/phuongnam/Documents/Ki2_nam3/Big_Data/hadoop-3.3.6"
import findspark
findspark.init()
import pyspark
# Creating a SparkSession in Python
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local")\
          .appName("Spark Streaming Demonstration")\
          .config("spark.some.config.option", "some-value")\
          .getOrCreate()
# keep the size of shuffles small
spark.conf.set("spark.sql.shuffle.partitions", "2") 

24/05/31 14:24:40 WARN Utils: Your hostname, dev-nam resolves to a loopback address: 127.0.1.1; using 192.168.1.14 instead (on interface wlp0s20f3)
24/05/31 14:24:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/31 14:24:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!ls 

 abs  'Big Data - Lab3.pdf'   src   taxi-data   test.py


## Word Count on NCat

In [4]:
folder = "../taxi-data/"
files = spark.read.format("csv").option("header", "false").load(folder)
files

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string]

In [5]:
files.show(5)

+------+---+-------------------+-------------------+---+-----+-------------------+------------------+---------------+------------------+-------------------+------------------+----+----+----+----+-----+----+----+-----+
|   _c0|_c1|                _c2|                _c3|_c4|  _c5|                _c6|               _c7|            _c8|               _c9|               _c10|              _c11|_c12|_c13|_c14|_c15| _c16|_c17|_c18| _c19|
+------+---+-------------------+-------------------+---+-----+-------------------+------------------+---------------+------------------+-------------------+------------------+----+----+----+----+-----+----+----+-----+
|yellow|  2|2015-12-01 18:35:53|2015-12-01 19:11:00|  1| 5.04|-74.015205383300781|40.715606689453125|              1|                 N|-73.979385375976562|40.756240844726563|   1|24.5|   1| 0.5| 5.26|   0| 0.3|31.56|
|yellow|  2|2015-12-01 19:02:06|2015-12-01 19:11:00|  1|  .70|-73.980827331542969|     40.7646484375|              1|           

In [3]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
# Create DataFrame representing the stream of input lines 
# from connection to localhost:50050
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 50050) \
    .load()
# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)
# Generate running word count
wordCounts = words.groupBy("word").count()
# Initialize ncat first
# ncat server: ncat -l -p 50050
# ncat client: ncat localhost 50050 (not the same with spark)
# Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()
query.awaitTermination(60)

24/05/31 08:57:48 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/05/31 08:57:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6c467f11-0332-4dc3-b539-1b99efdb8e22. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/05/31 08:57:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/05/31 08:57:49 ERROR MicroBatchExecution: Query [id = 05a67baf-6576-41fe-af85-92e231b1ddd2, runId = 7897a609-f4d4-43ad-ae16-5cfb92df85c2] terminated with error
java.net.ConnectException: Connection refused (Connection refused)
	at java.base/java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.base/java.net.Abstr

StreamingQueryException: [STREAM_FAILED] Query [id = 05a67baf-6576-41fe-af85-92e231b1ddd2, runId = 7897a609-f4d4-43ad-ae16-5cfb92df85c2] terminated with exception: Connection refused (Connection refused)

In [ ]:
# query.stop()

## Action-Time data
Reference: https://docs.databricks.com/en/_extras/notebooks/source/structured-streaming-python.html

In [ ]:
from pyspark.sql.types import *
# input path in local filesystem
inputPath = "."
# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
csvSchema = StructType([StructField("Action", StringType(), True), StructField("Time", LongType(), True)])

### Read the data in Batch mode

In [ ]:
staticInputDF = (
  spark
    .read
    .schema(csvSchema)          # Set the schema of the csv data
    .csv(inputPath, header=True)
)
# cast the Timestamp type since it is not automatically parsed
staticInputDF = staticInputDF.select('Action', (staticInputDF['Time']).cast("timestamp"))
# show the original data
staticInputDF.show(truncate=False)

In [ ]:
from pyspark.sql.functions import *      # for window() function
staticCountsDF = (
  staticInputDF
    .groupBy(
       staticInputDF.Action, 
       window(staticInputDF.Time, "1 hour"))    
    .count()
)
staticCountsDF.orderBy('window').show(truncate=False)

### Read the data in Streaming mode

In [ ]:
from pyspark.sql.functions import *      # for window() function
from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(csvSchema)                # Set the schema of the csv data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .csv(inputPath)
)
# cast the Timestamp type since it is not automatically parsed
streamingInputDF = streamingInputDF.select('Action', (streamingInputDF['Time']).cast("timestamp"))
# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.Action, 
      window(streamingInputDF.Time, "1 hour"))
    .count()
)
# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

In [ ]:
# This query prints the aggregation results to the console
query = (
  streamingCountsDF
    .writeStream
    .format("console")        # console or memory(= store in-memory table) 
    .outputMode("complete")   
    .option("truncate", "false")
    .start()
)
query.awaitTermination(60)

In [ ]:
# This query stores the aggregation results in memory then visualize it
query = (
  streamingCountsDF
    .writeStream
    .format("memory")         # console or memory(= store in-memory table) 
    .queryName("counts")      # counts = name of the in-memory table
    .outputMode("complete")   
    .option("truncate", "false")
    .start()
)
query.awaitTermination(60)

In [ ]:
query.stop()

In [ ]:
spark.sql('select * from counts order by window').show(truncate=False)